#  Exploring Types of MCP Servers and Agent Memory

### Welcome to Week 6 Day 3!

Let's experiment with a bunch more MCP Servers

In [19]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace
from agents.mcp import MCPServerStdio
import os
from IPython.display import Markdown, display
from datetime import datetime
load_dotenv(override=True)

True

🟢 **Type 1: Local MCP Server (Everything Runs Locally)**

🚀 **What is it?**
- A knowledge-graph based memory server.

🧠 **Key Features:**
1. **Persistent Memory**: Stores entities, their observations, and relationships between them.
2. **Knowledge Graph**: Organizes information in a graph structure for easy retrieval and reasoning.
3. **Local Operation**: Everything runs on your machine—no cloud required!

💡 **Why use it?**
- Empowers your agent with long-term memory.
- Enables storing and recalling facts, relationships, and context across conversations.

🔗 **Learn More:**  
  👉 https://github.com/modelcontextprotocol/servers/tree/main/src/memory


In [14]:
# Define the parameters required to launch the MCP memory server.
# - "command": The command to execute, here "npx" is used to run a Node.js package.
# - "args": The arguments for the command. "-y" auto-confirms prompts, and "mcp-memory-libsql" is the MCP memory server package.
# - "env": Environment variables for the server process. "LIBSQL_URL" specifies the SQLite database file to use for persistent storage.
params = {
    "command": "npx",
    "args": ["-y", "mcp-memory-libsql"],
    "env": {"LIBSQL_URL": "file:./memory/ed.db"}
}

# Start an asynchronous context manager to launch and manage the MCP memory server process.
# MCPServerStdio connects to the server using standard input/output pipes.
# - params: The server launch parameters defined above.
# - client_session_timeout_seconds: Timeout for client sessions (30 seconds here).
async with MCPServerStdio(params=params, client_session_timeout_seconds=30) as server:
    # Within the context, request the list of available MCP tools from the server.
    # This returns a list of tool objects that the server exposes for use.
    mcp_tools = await server.list_tools()

# Output the list of available MCP tools.
# This will display the tools that can be used to interact with the memory server.
mcp_tools

[Tool(name='create_entities', description='Create new entities with observations and optional embeddings', inputSchema={'type': 'object', 'properties': {'entities': {'type': 'array', 'items': {'type': 'object', 'properties': {'name': {'type': 'string'}, 'entityType': {'type': 'string'}, 'observations': {'type': 'array', 'items': {'type': 'string'}}, 'embedding': {'type': 'array', 'items': {'type': 'number'}, 'description': 'Optional vector embedding for similarity search'}}, 'required': ['name', 'entityType', 'observations']}}}, 'required': ['entities']}, annotations=None),
 Tool(name='search_nodes', description='Search for entities and their relations using text or vector similarity', inputSchema={'type': 'object', 'properties': {'query': {'oneOf': [{'type': 'string', 'description': 'Text search query'}, {'type': 'array', 'items': {'type': 'number'}, 'description': 'Vector for similarity search'}]}}, 'required': ['query']}, annotations=None),
 Tool(name='read_graph', description='Get 

In [15]:
instructions = "You use your entity tools as a persistent memory to store and recall information about your conversations."
request = "My name's Ed. I'm an LLM engineer. I'm teaching a course about AI Agents, including the incredible MCP protocol. \
MCP is a protocol for connecting agents with tools, resources and prompt templates, and makes it easy to integrate AI agents with capabilities."
model = "gpt-4.1-mini"

In [16]:
async with MCPServerStdio(params=params, client_session_timeout_seconds=30) as mcp_server:
    agent = Agent(name="agent", instructions=instructions, model=model, mcp_servers=[mcp_server])
    with trace("conversation"):
        result = await Runner.run(agent, request)
    display(Markdown(result.final_output))

Nice to meet you, Ed! It's great that you're teaching a course about AI Agents and covering the MCP protocol. If you'd like, I can help you with creating course materials, explanations, or examples related to AI Agents and the MCP protocol. Just let me know how I can assist!

In [ ]:
# In the previous cell, we set up an MCP memory server and used it to store some information about the user ("My name's Ed. I'm an LLM engineer...").
# That cell demonstrated how to persistently store information using the MCP protocol and the memory server.
# 
# In this cell, we are testing whether the memory server can recall information about the user that was previously stored.
# The code launches the MCP memory server again (with the same parameters, so it uses the same persistent database).
# It then creates an Agent with the same instructions and model as before, and connects it to the memory server.
# The agent is asked: "My name's Ed. What do you know about me?".
# The trace context ("conversation") is used to record the interaction for debugging or analysis.
# The result from the agent is displayed as Markdown, which should include any information the agent has recalled about "Ed" from the persistent memory.
async with MCPServerStdio(params=params, client_session_timeout_seconds=30) as mcp_server:
    agent = Agent(name="agent", instructions=instructions, model=model, mcp_servers=[mcp_server])
    with trace("conversation"):
        result = await Runner.run(agent, "My name's Ed. What do you know about me?")
    display(Markdown(result.final_output))

I know that you are Ed, an LLM engineer. You are also teaching a course about AI Agents. Is there anything specific you'd like to add or update about yourself?

### Check the trace:

https://platform.openai.com/traces

### The 2nd type of MCP server - runs locally, calls a web service

### Brave Search - apologies - this will need another API key! But it's free again.

https://brave.com/search/api/

Set up your account, and put your key in the .env under `BRAVE_API_KEY`

In [ ]:
# Set up the environment variable dictionary with the Brave Search API key.
# This will be passed to the MCP server process so it can authenticate requests to the Brave Search API.
env = {"BRAVE_API_KEY": os.getenv("BRAVE_API_KEY")}

# Define the parameters for launching the MCP server.
# The 'env' key in the params dictionary ensures the environment variable is available to the subprocess.
params = {
    "command": "npx",  # Use npx to run the MCP server for Brave Search
    "args": ["-y", "@modelcontextprotocol/server-brave-search"],  # Arguments to specify the MCP server package
    "env": env  # Pass the environment variable to the server process
}

async with MCPServerStdio(params=params) as server:
    mcp_tools = await server.list_tools()

mcp_tools

[Tool(name='brave_web_search', description='Performs a web search using the Brave Search API, ideal for general queries, news, articles, and online content. Use this for broad information gathering, recent events, or when you need diverse web sources. Supports pagination, content filtering, and freshness controls. Maximum 20 results per request, with offset for pagination. ', inputSchema={'type': 'object', 'properties': {'query': {'type': 'string', 'description': 'Search query (max 400 chars, 50 words)'}, 'count': {'type': 'number', 'description': 'Number of results (1-20, default 10)', 'default': 10}, 'offset': {'type': 'number', 'description': 'Pagination offset (max 9, default 0)', 'default': 0}}, 'required': ['query']}, annotations=None),
 Tool(name='brave_local_search', description="Searches for local businesses and places using Brave's Local Search API. Best for queries related to physical locations, businesses, restaurants, services, etc. Returns detailed information including:\

In [21]:
instructions = "You are able to search the web for information and briefly summarize the takeaways."
request = f"Please research the latest news on Amazon stock price and briefly summarize its outlook. \
For context, the current date is {datetime.now().strftime('%Y-%m-%d')}"
model = "gpt-4o-mini"

In [22]:
async with MCPServerStdio(params=params, client_session_timeout_seconds=30) as mcp_server:
    agent = Agent(name="agent", instructions=instructions, model=model, mcp_servers=[mcp_server])
    with trace("conversation"):
        result = await Runner.run(agent, request)
    display(Markdown(result.final_output))

As of July 2025, Amazon's stock price shows a generally optimistic outlook based on the latest developments:

1. **Current Price and Performance**:
   - Amazon's stock is projected to begin July 2025 at **$220**.
   - Forecasts suggest a potential maximum of **$255** and a minimum of **$199**, leading to an average price of **$228** for the month.

2. **Long-Term Projections**:
   - Predictive models indicate that Amazon's stock could see an increase from **$447** to **$563** by 2032, marking a growth potential of around **+26%** over the next few years.

3. **Recent Analyst Ratings**:
   - Truist recently raised its price target for Amazon from **$226** to **$250**, maintaining a "Buy" rating, reflecting positive sentiment among analysts.

4. **Market Capitalization**:
   - Amazon's market capitalization stands at approximately **$2.33 trillion**, with a slight increase of **1.37%** in the past week.

5. **Upcoming Earnings Report**:
   - Investors are awaiting the next earnings report, which is scheduled for **July 31, 2025**, a pivotal event that may influence stock performance.

Overall, the outlook for Amazon's stock appears favorable, supported by positive analyst ratings and market expectations.

### As usual, check out the trace:

https://platform.openai.com/traces

## 🌐 Third Type: Running MCP Servers Remotely

### What is a "Remote MCP Server"? 🤔
- Also called **hosted MCP server** or **managed MCP server**
- These are MCP servers that run on someone else's infrastructure, not your local machine.

### Availability & Discovery 🔍
- Finding a public or managed remote MCP server is **rare**.
- There is **no standard directory** or discovery mechanism for public MCP servers.

### Who Offers Remote MCP Servers? 🏢
- **Anthropic**: Lists some remote MCP servers, but these are for **paid, business users** only.
  - 📄 [Anthropic Remote MCP Servers Documentation](https://docs.anthropic.com/en/docs/agents-and-tools/remote-mcp-servers)

- **Cloudflare**: Provides tooling to **create and deploy your own** remote MCP servers.
  - ⚙️ [Cloudflare Remote MCP Server Guide](https://developers.cloudflare.com/agents/guides/remote-mcp-server/)
  - Note: This is not a common practice yet.

### Summary 📝
- Remote MCP servers are possible, but **not common** or widely available for general use.
- Most users will run MCP servers **locally** or within their own cloud infrastructure.


# And back to the 2nd type: the Polygon.io MCP Server

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 90px; height: 90px; vertical-align: middle; font-size: 80px; text-align: center;">
            🛑🛑<br>
            <span style="font-size: 40px;">🚦</span>
        </td>
        <td>
            <h2 style="color:#ff7800;">PLEASE READ!!-</h2>
            <span style="color:#ff7800;">This service for financial market data has both a FREE plan and a PAID plan, and we can use either depending on your appetite.
            </span>
        </td>
    </tr>
</table>

## NEW SECTION: Introducing polygon.io

Polygon.io is a hugely popular financial data provider. It has a free plan and a paid plan. And it also has an MCP Server!

First, read up on polygon.io on their excellent website, including looking at their pricing:

https://polygon.io

### Polygon.io Part 1: Polygon.io free service (the paid will be totally optional, of course!)

1. Please sign up for polygon.io (top right)  
2. Once signed in, please select "Keys" in the left hand navigation
3. Press the blue "New Key" button
4. Copy the key name
5. Edit your .env file and add the row:

`POLYGON_API_KEY=xxxx`

In [23]:
load_dotenv(override=True)
polygon_api_key = os.getenv("POLYGON_API_KEY")
if not polygon_api_key:
    print("POLYGON_API_KEY is not set")

In [24]:
from polygon import RESTClient
client = RESTClient(polygon_api_key)
client.get_previous_close_agg("AAPL")[0]

PreviousCloseAgg(ticker='AAPL', close=211.14, high=211.33, low=207.22, open=209.53, timestamp=1752091200000, volume=48749367.0, vwap=209.5626)

### Wrapped into a python module that caches end of day prices

I've made a python module `market.py` that uses this API to look up share prices.

But the free API is quite heavily rate limited - so I've been a bit sneaky; when you ask for a share price, this function retrieves the entire end-of-day equity market, and caches it in our database.


In [25]:
from market import get_share_price
get_share_price("AAPL")

211.14

In [27]:
# no rate limiting concerns!

for i in range(1000):
    get_share_price("AAPL")
get_share_price("AAPL")

211.14

### And I've made this into an MCP Server

Just as we did with accounts.py; see `market_server.py`

In [28]:
params = {"command": "uv", "args": ["run", "market_server.py"]}
async with MCPServerStdio(params=params) as server:
    mcp_tools = await server.list_tools()
mcp_tools

[Tool(name='lookup_share_price', description='This tool provides the current price of the given stock symbol.\n\n    Args:\n        symbol: the symbol of the stock\n    ', inputSchema={'properties': {'symbol': {'title': 'Symbol', 'type': 'string'}}, 'required': ['symbol'], 'title': 'lookup_share_priceArguments', 'type': 'object'}, annotations=None)]

### Let's try it out!

Hopefully gpt-4o-mini is smart enough to know that the symbol for Apple is AAPL

In [29]:
instructions = "You answer questions about the stock market."
request = "What's the share price of Apple?"
model = "gpt-4.1-mini"

async with MCPServerStdio(params=params) as mcp_server:
    agent = Agent(name="agent", instructions=instructions, model=model, mcp_servers=[mcp_server])
    with trace("conversation"):
        result = await Runner.run(agent, request)
    display(Markdown(result.final_output))

The current share price of Apple (AAPL) is $211.14.

## Polygon.io Part 2: Paid Plan - Totally Optional!

If you are interested, you can subscribe to the monthly plan to get more up to date market data, and unlimited API calls.

If you do wish to do this, then it also makes sense to use the full MCP server that Polygon.io has released, to take advantage of all their functionality.



In [ ]:

params = {"command": "uvx",
          "args": ["--from", "git+https://github.com/polygon-io/mcp_polygon@v0.1.0", "mcp_polygon"],
          "env": {"POLYGON_API_KEY": polygon_api_key}
          }
async with MCPServerStdio(params=params) as server:
    mcp_tools = await server.list_tools()
mcp_tools


### Wow that's a lot of tools!

Let's try them out - hopefully the sheer number of tools doesn't overwhelm gpt-4o-mini!

With the $29 monthly plan, we don't have access to some of the APIs, so I've needed to specify which APIs can be called.

If you've splashed out on a bigger plan, feel free to remove my extra constraint..

In [ ]:
instructions = "You answer questions about the stock market."
request = "What's the share price of Apple? Use your get_snapshot_ticker tool to get the latest price."
model = "gpt-4.1-mini"

async with MCPServerStdio(params=params) as mcp_server:
    agent = Agent(name="agent", instructions=instructions, model=model, mcp_servers=[mcp_server])
    with trace("conversation"):
        result = await Runner.run(agent, request)
    display(Markdown(result.final_output))

## Setting up your .env file

If you do decide to have a paid plan, please add this to your .env file to indicate:

`POLYGON_PLAN=paid`

And if you decide to go all the way for the realtime API, then please do:

`POLYGON_PLAN=realtime`

In [ ]:
load_dotenv(override=True)

polygon_plan = os.getenv("POLYGON_PLAN")
is_paid_polygon = polygon_plan == "paid"
is_realtime_polygon = polygon_plan == "realtime"

if is_paid_polygon:
    print("You've chosen to subscribe to the paid Polygon plan, so the code will look at prices on a 15 min delay")
elif is_realtime_polygon:
    print("Wowzer - you've chosen to subscribe to the realtime Polygon plan, so the code will look at realtime prices")
else:
    print("According to your .env file, you've chosen to subscribe to the free Polygon plan, so the code will look at EOD prices")